In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import os.path
from datetime import datetime
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

train_path = '/home/user/data/data/ML/train'
validation_path = '/home/user/data/test/'

In [2]:
# Checking the image sizes
img = Image.open('/home/user/data/data/ML/train/target_1/tumor_0_9_9178.jpg')
print(img.size)

image_size = (180,180)
batch_size = 32
epochs = 20
eval_all, mdl_lst, aug_lst = [], [], []

(224, 224)


In [3]:
class CustomAugment(object):
    def __call__(self, image):        
        # Random flips and grayscale with some stochasticity
        img = self._random_apply(tf.image.flip_left_right, image, p=0.6)
        img = self._random_apply(self._color_drop, img, p=0.9)
        return img

    def _color_drop(self, x):
        image = tf.image.rgb_to_grayscale(x)
        image = tf.tile(x, [1, 1, 1, 3])
        return x
    
    def _random_apply(self, func, x, p):
        return tf.cond(
          tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                  tf.cast(p, tf.float32)),
          lambda: func(x),
          lambda: x)

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_path,
    labels="inferred",
    label_mode="binary",
    validation_split=0.2,
    subset="training",
    seed=0,
    image_size=image_size,
    batch_size=batch_size,
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=validation_path,
    labels="inferred",
    label_mode="binary",
    shuffle=False,
    seed=0,
    image_size=image_size,
    batch_size=batch_size,
)

Found 5000 files belonging to 2 classes.
Using 4000 files for training.


2021-11-10 23:56:13.181796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-10 23:56:13.196169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-10 23:56:13.197185: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-10 23:56:13.199009: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 6000 files belonging to 2 classes.


In [5]:
train_ds = train_ds.prefetch(buffer_size=batch_size)

In [6]:
def gen_augmentation(input_shape):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)
    return x, inputs

In [7]:
def make_basic_model(num_classes, x, inputs):
    
    # Entry block
    x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

In [8]:
def make_effB3_model(x, inputs):
    
    effB3_model = tf.keras.applications.EfficientNetB3(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=image_size + (3,),
    include_top=False)  # Do not include the ImageNet classifier at the top.

    effB3_model.trainable = False
    
    x = layers.Rescaling(1.0 / 255)(x)
    x = effB3_model(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)

In [9]:
def make_vgg16_model(x, inputs):
    
    vgg16_model = tf.keras.applications.VGG16(
        weights="imagenet",
        input_shape=image_size + (3,),
        include_top=False)
    vgg16_model.trainable = False
    x = layers.Rescaling(1.0 / 255)(x)
    x = vgg16_model(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)

In [10]:
def make_rn152v2_model(x, inputs):
    
    rn152v2_model = tf.keras.applications.ResNet152V2(
        weights="imagenet",
        input_shape=image_size + (3,),
        include_top=False)
    rn152v2_model.trainable = False
    x = layers.Rescaling(1.0 / 255)(x)
    x = rn152v2_model(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)

In [11]:
def make_dn201_model(x, inputs):
    
    dn201_model = tf.keras.applications.DenseNet201(
        weights="imagenet",
        input_shape=image_size + (3,),
        include_top=False)
    dn201_model.trainable = False
    x = layers.Rescaling(1.0 / 255)(x)
    x = dn201_model(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)

In [12]:
def execute_model(model, train_ds, test_ds, group, seq, mod_type):

    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()],
    )
    model.fit(train_ds, epochs=epochs)
    eval_lst = [seq]+model.evaluate(train_ds)+model.evaluate(test_ds)+[group, mod_type]
    
    return model, eval_lst

In [13]:
group_lst = ["flip_rotation", "flip_color_rot", "flip_color_rot_zoom", 
             "flip_color_rot_zoom_gn", "flip_color_rot_zoom_cont", "flipHV_color_rot_zoom_cont"]

mod_type = ["basic", "effB3", "vgg16", "resnet152v2", "denseNet201"]

aug_lst.append(keras.Sequential([layers.RandomFlip("horizontal"),
                                 layers.RandomRotation(0.1)]))

aug_lst.append(keras.Sequential([layers.Lambda(CustomAugment()),
                                 layers.RandomRotation(0.1)]))

aug_lst.append(keras.Sequential([layers.Lambda(CustomAugment()),
                                 layers.RandomRotation(0.1),
                                 layers.RandomZoom(0.1)]))

aug_lst.append(keras.Sequential([layers.Lambda(CustomAugment()),
                                 layers.RandomRotation(0.1),
                                 layers.RandomZoom(0.1),
                                 layers.GaussianNoise(0.1)]))

aug_lst.append(keras.Sequential([layers.Lambda(CustomAugment()),
                                 layers.RandomRotation(0.1),
                                 layers.RandomZoom(0.1),
                                 layers.GaussianNoise(0.1),
                                 layers.RandomContrast(0.2)]))

aug_lst.append(keras.Sequential([layers.RandomRotation(0.1),
                                 layers.RandomZoom(0.1),
                                 layers.GaussianNoise(0.1),
                                 layers.RandomContrast(0.2),
                                 layers.RandomFlip("horizontal_and_vertical")]))

In [ ]:
k=0
starts_ends = []
start_time = time.time()

for i in range(0, len(aug_lst)):
    for j in range(0, len(mod_type)):
        print("Model: " + str(k))
        data_augmentation = aug_lst[i]
        aug_mod, aug_inputs = gen_augmentation(image_size + (3,))
        if (j==0):
            model = make_basic_model(num_classes=2, x=aug_mod, inputs=aug_inputs)
        elif (j==1):
            model = make_effB3_model(x=aug_mod, inputs=aug_inputs)
        elif (j==2):
            model = make_vgg16_model(x=aug_mod, inputs=aug_inputs)
        elif (j==3):
            model = make_rn152v2_model(x=aug_mod, inputs=aug_inputs)
        else:
            model = make_dn201_model(x=aug_mod, inputs=aug_inputs)
        mdl, eval_lst = execute_model(model, train_ds, test_ds, group_lst[i], k, mod_type[j])
        mdl_lst.append(mdl)
        eval_all.append(eval_lst)
        k += 1
        print()
        
tot_in_sec = time.time() - start_time
print("--- %s seconds ---" % (tot_in_sec))

Model: 0
Epoch 1/20


2021-11-10 23:56:16.601992: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-10 23:56:17.792189: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204


125/125 [==============================] - 26s 172ms/step - loss: 0.3751 - binary_accuracy: 0.8530 - auc: 0.8857
Epoch 2/20
125/125 [==============================] - 22s 173ms/step - loss: 0.3140 - binary_accuracy: 0.8865 - auc: 0.9108
Epoch 3/20
125/125 [==============================] - 22s 173ms/step - loss: 0.3133 - binary_accuracy: 0.8895 - auc: 0.9104
Epoch 4/20
125/125 [==============================] - 22s 172ms/step - loss: 0.2705 - binary_accuracy: 0.9018 - auc: 0.9308
Epoch 5/20
125/125 [==============================] - 22s 173ms/step - loss: 0.2623 - binary_accuracy: 0.9035 - auc: 0.9307
Epoch 6/20
125/125 [==============================] - 22s 172ms/step - loss: 0.2602 - binary_accuracy: 0.9072 - auc: 0.9314
Epoch 7/20
125/125 [==============================] - 22s 172ms/step - loss: 0.2387 - binary_accuracy: 0.9170 - auc: 0.9434
Epoch 8/20
125/125 [==============================] - 22s 172ms/step - loss: 0.2521 - binary_accuracy: 0.9053 - auc: 0.9400
Epoch 9/20
125/125 

234561536/234545216 [==============================] - 7s 0us/step
Epoch 1/20
125/125 [==============================] - 22s 103ms/step - loss: 8.2562 - binary_accuracy: 0.7240 - auc_3: 0.6728
Epoch 2/20
125/125 [==============================] - 13s 103ms/step - loss: 4.1234 - binary_accuracy: 0.7355 - auc_3: 0.69882s - loss: 4.2303 - binary_accurac
Epoch 3/20
125/125 [==============================] - 13s 103ms/step - loss: 5.7203 - binary_accuracy: 0.7245 - auc_3: 0.67312s - loss: 4.9896 - binary_accuracy: 0.7381 - auc_3: - ETA: 2s - loss: 4.9740 - binary_accuracy: 
Epoch 4/20
125/125 [==============================] - 13s 103ms/step - loss: 4.8457 - binary_accuracy: 0.7673 - auc_3: 0.72642s - loss: 5.0658 - binary_accurac
Epoch 5/20
125/125 [==============================] - 13s 103ms/step - loss: 4.7734 - binary_accuracy: 0.7470 - auc_3: 0.7126
Epoch 6/20
125/125 [==============================] - 13s 103ms/step - loss: 3.0122 - binary_accuracy: 0.7925 - auc_3: 0.7682
Epoch 7/20
1

In [ ]:
cols=["id", "train_loss", "train_bin_acc", "train_auc", "test_loss", "test_bin_acc", "test_auc", "group", "model"]
df_best_model = pd.DataFrame(eval_all,columns=cols)
df_best_model = df_best_model.sort_values("test_auc", ascending=False).reset_index(drop=True)

In [ ]:
df_best_model

In [ ]:
mdl = mdl_lst[df_best_model.loc[0, "id"]]

In [ ]:
file_paths = test_ds.file_paths

def generate_sub_best_model(model, file_paths, save_sub, dtn):
    predictions = np.array([])
    labels =  np.array([])
    for x, y in test_ds:
        predictions = np.concatenate([predictions, model.predict(x).ravel()])
        labels = np.concatenate([labels, y.numpy().ravel()])
    df_solution = pd.DataFrame(data={'file_paths': file_paths, 'predictions': labels})
    #df_solution.to_csv('df_solution.csv', index=False)
    df_submission = pd.DataFrame(data={'file_paths': file_paths, 'predictions': predictions})
    df_submission["file_paths"] = df_submission["file_paths"].apply(lambda x: 
                                                                    x.replace("/home/user/data/test",
                                                                              "/data/challenges_data/test"))
    
    if (save_sub):
        df_submission.to_csv('df_submission_'+str(dtn)+'.csv', index=False)
        model.save("best_model"+dtn+".h5")
    return df_submission

In [ ]:
save_submissions = True

In [ ]:
dtn = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
df_submission = generate_sub_best_model(mdl, file_paths, save_submissions, dtn)

In [ ]:
df_submission.head()